# Hands on Iceberg with AWS Athena

Before you go through this notebook, please check [README](./README.md) for a complete guide on how to set up everything.

This notebook tries to help you to try AWS Athena on a easy a reproducible way.

In [1]:
# Initial Run
%env AWS_PROFILE=given_profile_name

import os

from dotenv import load_dotenv

from magic_cells import get_var

if not load_dotenv(f"{os.getcwd()}/terraform.env"):
    raise ImportError("terraform.env not does not exist. You might need to run terraform first.")

DATA_BUCKET = os.environ["DATA_BUCKET"]
print(f"{DATA_BUCKET = }")


env: AWS_PROFILE=illo
DATA_BUCKET = 'illo-laz-iceberg-data'


In [ ]:
# Only run this cell once to upload the CSV to the S3 bucket
import requests
from io import BytesIO
from zipfile import ZipFile

import boto3

DATASET_URL = "https://github.com/awesomedata/awesome-public-datasets/raw/master/Datasets/titanic.csv.zip"
CSV_FOLDER = "csv/"

# Download the dataset and open the zip file
response_bytes = BytesIO(requests.get(DATASET_URL).content)
zip_file = ZipFile(response_bytes)

# Upload CSV file to S3 Bucket
s3 = boto3.client("s3")
print(f"\nUploading files to '{DATA_BUCKET}/{CSV_FOLDER}'")

for file in zip_file.namelist():
    if not file.startswith("__MACOSX") and file.endswith(".csv"):
        csv_file = zip_file.open(file)
        s3.upload_fileobj(csv_file, DATA_BUCKET, CSV_FOLDER + file)
        print(f"    file '{file}' uploaded successfully.")

### Notebook with magic

I created some magic cell commands that will make this notebook more enjoyable.

- athena_exec: Its intended to be a function to edit your table or table schema. You could either create a table, delete it, populate or any action that requires any write. Its usage is very simple: `%%athena_exec <your_database>`

- athena: Allows you to read data from Athena using SQL into a python dataframe.<br/>
The usage will be `%%athena <your_database> <(Optional) python_variable_name>`

Find examples of usage in the three following cells.

In [ ]:
%%athena_exec iceberg_catalog
-- This query will create a Glue table pointing to our CSVs files
CREATE EXTERNAL TABLE passenger_data_csv
(
    PassengerId bigint,
    Survived bigint,
    Pclass bigint,
    Name string,
    Sex string,
    Age double,
    SibSp bigint,
    Parch bigint,
    Ticket string,
    Fare double,
    Cabin string,
    Embarked string
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 's3://{DATA_BUCKET}/csv/';


In [ ]:
%%athena iceberg_catalog df
-- Simple select that saves the result as Pandas dataframe in the variable you defined
SELECT * FROM passenger_data_csv LIMIT 10

In [ ]:
df = get_var('df')
df

## Working with Iceberg

Here you have two queries. On the first one, you define the table, where to store the data and metadata and the structure of it.
The second one, uses the previous data from a CSV file to insert those values in there.
You can go to your s3 bucket and you'll see all the files being created there following the Iceberg specs.


In [ ]:
%%athena_exec iceberg_catalog
-- This query will create a Glue table for Iceberg
CREATE TABLE passenger_data_iceberg
(
    PassengerId bigint,
    Survived bigint,
    Pclass bigint,
    Name string,
    Sex string,
    Age double,
    SibSp bigint,
    Parch bigint,
    Ticket string,
    Fare double,
    Cabin string,
    Embarked string
)
LOCATION 's3://{DATA_BUCKET}/passenger_iceberg/'
TBLPROPERTIES ( 'table_type' ='ICEBERG'  );


In [ ]:
%%athena_exec iceberg_catalog
-- Load data into the iceberg table from csv table
INSERT INTO passenger_data_iceberg
SELECT * FROM passenger_data_csv ;